<a href="https://colab.research.google.com/github/dinithihewa/Dinithi163.github.io/blob/main/ML_Model_new_Training_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm.libsvm import predict_proba
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import pickle

In [ ]:
def col_summury(data_table):
    print('The table size is {}'.format(data_table.shape))
    print('\n')
    col_names=list(data_table.columns)
    for x in col_names:
        print('column name is : {}'.format(x))
        print(data_table[x].unique())
        print('Unique count is :{}'.format(data_table[x].nunique()))
        print('\n')

In [ ]:
Data_path='DATA SET/DATA SET/archive (4)/uci-open-university-learning-analytics-dataset'

In [ ]:
studInfo=pd.read_csv(Data_path+ "/studentInfo.csv")
assessments=pd.read_csv(Data_path+"/assessments.csv")
studAssement=pd.read_csv(Data_path+"/studentAssessment.csv")
studentRegistration = pd.read_csv(Data_path+'/studentRegistration.csv')
studVle=pd.read_csv(Data_path+"/studentVle.csv")
vle=pd.read_csv(Data_path+"/vle.csv")
courses = pd.read_csv(Data_path+'/courses.csv')

### Merge studInfo and courses

In [ ]:
studInfo=studInfo[studInfo["final_result"]!="Withdrawn"]

In [ ]:
df_merge_1=pd.merge(studInfo,courses,how='left',on=['code_module','code_presentation'])

In [ ]:
df_merge_1.shape,studInfo.shape,courses.shape

((22437, 13), (22437, 12), (22, 3))

In [ ]:
df_merge_1.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,module_presentation_length
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,268
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,268
2,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,268
3,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,268
4,AAA,2013J,38053,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,268


In [ ]:
# Calculate average clicks of students on VLE activity sites.

clicks = studVle.groupby(['id_student', 'code_module', 'code_presentation']).agg({'sum_click':'mean'})
clicks.reset_index(level=[0, 1, 2], inplace=True)

df_merge_2 = pd.merge(df_merge_1,
                        clicks,
                        how = 'left',
                        left_on = ['id_student', 'code_module', 'code_presentation'],
                        right_on = ['id_student', 'code_module', 'code_presentation'])

In [ ]:
df_merge_2.shape,df_merge_1.shape,clicks.shape

((22437, 14), (22437, 13), (29228, 4))

In [ ]:
df_merge_2.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,module_presentation_length,sum_click
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,268,4.765306
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,268,3.337209
2,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,268,3.254902
3,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,268,2.937500
4,AAA,2013J,38053,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,268,3.381743


In [ ]:
assessments_totalled = pd.merge(studAssement,
                        assessments,
                        how = 'left',
                        left_on = ['id_assessment'],
                        right_on = ['id_assessment'])
assessments_totalled=assessments_totalled[assessments_totalled['date']!='?']
assessments_totalled['date']=assessments_totalled['date'].astype(int)
assessments_totalled['delay'] = assessments_totalled['date_submitted'] - assessments_totalled['date']

In [ ]:
student_delays = assessments_totalled.groupby(['code_presentation', 'code_module', 'id_student']).agg({'delay':'mean'})
student_delays.reset_index(level=[0, 1, 2], inplace=True)

df_merge_3 = pd.merge(df_merge_2,
                        student_delays,
                        how = 'left',
                        left_on = ['id_student', 'code_module', 'code_presentation'],
                        right_on = ['id_student', 'code_module', 'code_presentation'])

In [ ]:
df_merge_3.shape,df_merge_2.shape,student_delays.shape

((22437, 15), (22437, 14), (25842, 4))

In [ ]:
df_merge_4 = pd.merge(df_merge_3,
                        studentRegistration,
                        how = 'left',
                        left_on = ['id_student', 'code_module', 'code_presentation'],
                        right_on = ['id_student', 'code_module', 'code_presentation'])

In [ ]:
df_merge_4.shape,df_merge_3.shape,studentRegistration.shape

((22437, 17), (22437, 15), (32593, 5))

In [ ]:
model_data = df_merge_4[[
#'id_student',
'code_module',
'code_presentation',
'gender',
# 'region',
'highest_education',
# 'imd_band',
'age_band',
'num_of_prev_attempts',
'studied_credits',
'disability',
'sum_click',
'delay',
# 'date_registration',
'final_result'
]]

model_data2 = model_data.copy()

In [ ]:
model_data2.head()

,code_module,code_presentation,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,delay,final_result
0,AAA,2013J,M,HE Qualification,55<=,0,240,N,4.765306,-1.8,Pass
1,AAA,2013J,F,HE Qualification,35-55,0,60,N,3.337209,0.0,Pass
2,AAA,2013J,F,A Level or Equivalent,35-55,0,60,N,3.254902,-2.0,Pass
3,AAA,2013J,F,Lower Than A Level,0-35,0,60,N,2.937500,11.4,Pass
4,AAA,2013J,M,A Level or Equivalent,35-55,0,60,N,3.381743,2.0,Pass


In [ ]:
model_data2[model_data2.final_result=='Fail']

,code_module,code_presentation,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,delay,final_result
16,AAA,2013J,M,A Level or Equivalent,35-55,0,150,N,1.611111,1.500000,Fail
44,AAA,2013J,F,A Level or Equivalent,0-35,0,60,Y,3.109375,4.800000,Fail
45,AAA,2013J,M,Lower Than A Level,0-35,0,120,N,2.679825,2.500000,Fail
58,AAA,2013J,F,A Level or Equivalent,0-35,0,180,N,2.760000,16.200000,Fail
63,AAA,2013J,M,HE Qualification,0-35,0,120,N,4.313131,5.000000,Fail
...,...,...,...,...,...,...,...,...,...,...,...
22415,GGG,2014J,F,Lower Than A Level,35-55,0,30,N,2.839844,-76.500000,Fail
22426,GGG,2014J,F,A Level or Equivalent,35-55,1,30,N,3.066667,-7.875000,Fail
22427,GGG,2014J,F,Lower Than A Level,0-35,0,30,Y,3.153959,-63.333333,Fail
22428,GGG,2014J,F,Lower Than A Level,35-55,0,90,N,1.125000,NaN,Fail


In [ ]:
cols=model_data2.columns
for col in cols:
    print(col)
    print(model_data2[col].unique())
    print('------')

code_module
['AAA' 'BBB' 'CCC' 'DDD' 'EEE' 'FFF' 'GGG']
------
code_presentation
['2013J' '2014J' '2013B' '2014B']
------
gender
['M' 'F']
------
highest_education
['HE Qualification' 'A Level or Equivalent' 'Lower Than A Level'
 'Post Graduate Qualification' 'No Formal quals']
------
age_band
['55<=' '35-55' '0-35']
------
num_of_prev_attempts
[0 1 2 4 3 5 6]
------
studied_credits
[240  60 120  90 150 180 420 170  80 300 210  75 135  70 225 325 130 195
 105 330 270 165 250  30  40  45 160 145 100 630  50 220 110  95 280 155
 190 200 140  85 215  55 115  65 310 360 430]
------
disability
['N' 'Y']
------
sum_click
[4.76530612 3.3372093  3.25490196 ... 2.62983425 3.76793249 3.40331492]
------
delay
[ -1.8          0.          -2.         ... -42.85714286  -7.875
 -50.42857143]
------
final_result
['Pass' 'Fail' 'Distinction']
------


In [ ]:
# model_data2['final_result'] = model_data2['final_result'].map(dict(Pass=1, Distinction=2, Fail = 0, Withdrawn = -1)) # old_one
model_data2['final_result'] = model_data2['final_result'].map(dict(Pass=1, Distinction=2, Fail = 0, Withdrawn = 3)) # new_one

# model_data2.gender = model_data2.gender.map(dict(M=1, F=0))
# model_data2.disability = model_data2.disability.map(dict(Y=1, N=0))
# model_data2.disability = model_data2.disability.map(dict(Y=1, N=0))
# model_data2.disability = model_data2.disability.map(dict(Y=1, N=0))
# model_data2.disability = model_data2.disability.map(dict(Y=1, N=0))
# model_data2.disability = model_data2.disability.map(dict(Y=1, N=0))
# model_data2.disability = model_data2.disability.map(dict(Y=1, N=0))
# model_data2.disability = model_data2.disability.map(dict(Y=1, N=0))
# model_data2.disability = model_data2.disability.map(dict(Y=1, N=0))
# model_data2.disability = model_data2.disability.map(dict(Y=1, N=0))

In [ ]:
le = LabelEncoder()

model_data2['gender'] = le.fit_transform(model_data2['gender'])
model_data2['disability'] = le.fit_transform(model_data2['disability'])
# model_data2['region'] = le.fit_transform(model_data2['region'])
model_data2['highest_education'] = le.fit_transform(model_data2['highest_education'])
model_data2['age_band'] = le.fit_transform(model_data2['age_band'])
model_data2['code_presentation'] = le.fit_transform(model_data2['code_presentation'])
model_data2['code_module'] = le.fit_transform(model_data2['code_module'])

In [ ]:
#Clean data from 'na' values
model_data2.sum_click = model_data2.sum_click.fillna(0)
# model_data2.date_registration = model_data2.date_registration.fillna(0)
model_data2.delay = model_data2.delay.fillna(0)

In [ ]:
model_data2.head()

,code_module,code_presentation,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,delay,final_result
0,0,1,1,1,2,0,240,0,4.765306,-1.8,1
1,0,1,0,1,1,0,60,0,3.337209,0.0,1
2,0,1,0,0,1,0,60,0,3.254902,-2.0,1
3,0,1,0,2,0,0,60,0,2.937500,11.4,1
4,0,1,1,0,1,0,60,0,3.381743,2.0,1


In [ ]:
model_data2.tail()

,code_module,code_presentation,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,delay,final_result
22432,6,3,0,0,0,0,30,1,2.629834,-31.444444,2
22433,6,3,0,2,0,0,30,0,2.157895,0.000000,0
22434,6,3,0,2,1,0,30,0,3.767932,-36.222222,2
22435,6,3,0,0,0,0,30,1,2.888889,-45.333333,1
22436,6,3,0,1,1,0,30,0,3.403315,-50.428571,2


In [ ]:
# model_data2=model_data2[model_data2.date_registration!='?']

In [ ]:
col_summury(model_data2)

The table size is (22437, 11)


column name is : code_module
[0 1 2 3 4 5 6]
Unique count is :7


column name is : code_presentation
[1 3 0 2]
Unique count is :4


column name is : gender
[1 0]
Unique count is :2


column name is : highest_education
[1 0 2 4 3]
Unique count is :5


column name is : age_band
[2 1 0]
Unique count is :3


column name is : num_of_prev_attempts
[0 1 2 4 3 5 6]
Unique count is :7


column name is : studied_credits
[240  60 120  90 150 180 420 170  80 300 210  75 135  70 225 325 130 195
 105 330 270 165 250  30  40  45 160 145 100 630  50 220 110  95 280 155
 190 200 140  85 215  55 115  65 310 360 430]
Unique count is :47


column name is : disability
[0 1]
Unique count is :2


column name is : sum_click
[4.76530612 3.3372093  3.25490196 ... 2.62983425 3.76793249 3.40331492]
Unique count is :18749


column name is : delay
[ -1.8          0.          -2.         ... -42.85714286  -7.875
 -50.42857143]
Unique count is :2806


column name is : final_result
[1 0

In [ ]:
final_df=model_data2.copy()

In [ ]:
final_df.head(5)

,code_module,code_presentation,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,delay,final_result
0,0,1,1,1,2,0,240,0,4.765306,-1.8,1
1,0,1,0,1,1,0,60,0,3.337209,0.0,1
2,0,1,0,0,1,0,60,0,3.254902,-2.0,1
3,0,1,0,2,0,0,60,0,2.937500,11.4,1
4,0,1,1,0,1,0,60,0,3.381743,2.0,1


In [ ]:
final_df.dtypes

code_module               int32
code_presentation         int32
gender                    int32
highest_education         int32
age_band                  int32
num_of_prev_attempts      int64
studied_credits           int64
disability                int32
sum_click               float64
delay                   float64
final_result              int64
dtype: object

In [ ]:
#Splitting the data into independent and dependent variables
X = final_df.iloc[:,0:10].values
y = final_df.iloc[:,10].values
print('----------------------------------------')
print('The independent features set: ')
print('----------------------------------------')
print(X[:5,:])

print('----------------------------------------')
print('The dependent variable: ')
print('----------------------------------------')
print(y[:5])

----------------------------------------
The independent features set: 
----------------------------------------
[[  0.           1.           1.           1.           2.
    0.         240.           0.           4.76530612  -1.8       ]
 [  0.           1.           0.           1.           1.
    0.          60.           0.           3.3372093    0.        ]
 [  0.           1.           0.           0.           1.
    0.          60.           0.           3.25490196  -2.        ]
 [  0.           1.           0.           2.           0.
    0.          60.           0.           2.9375      11.4       ]
 [  0.           1.           1.           0.           1.
    0.          60.           0.           3.38174274   2.        ]]
----------------------------------------
The dependent variable: 
----------------------------------------
[1 1 1 1 1]


In [ ]:
# Creating the Training and Test set from data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 21)

In [ ]:
X_train.shape,y_train.shape

((17949, 10), (17949,))

In [ ]:
X_test.shape,y_test.shape

((4488, 10), (4488,))

In [ ]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

## Random Forest Classification

In [ ]:
# Fitting Random Forest Classification to the Training set
classifier_rfc = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier_rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [ ]:
# Predicting the Test set results
y_pred_rfc = classifier_rfc.predict(X_test)

In [ ]:
print ("Train Accuracy=",accuracy_score(y_train, classifier_rfc.predict(X_train)))
print ("Test Accuracy=", round(accuracy_score(y_test, y_pred_rfc), 2))

Train Accuracy= 0.978494623655914
Test Accuracy= 0.59


In [ ]:
print(confusion_matrix(y_test, y_pred_rfc))

[[ 656  751   50]
 [ 361 1912  179]
 [  59  436   84]]


In [ ]:
print(classification_report(y_test, y_pred_rfc))

              precision    recall  f1-score   support

           0       0.61      0.45      0.52      1457
           1       0.62      0.78      0.69      2452
           2       0.27      0.15      0.19       579

    accuracy                           0.59      4488
   macro avg       0.50      0.46      0.47      4488
weighted avg       0.57      0.59      0.57      4488



## Gradient Boosting Classifier

In [ ]:
classifier_gb = GradientBoostingClassifier(random_state=0)

In [ ]:
classifier_gb.fit(X_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
# Predicting the Test set results
y_pred_gb = classifier_gb.predict(X_test)

In [ ]:
print ("Train Accuracy=",accuracy_score(y_train, classifier_gb.predict(X_train)))
print ("Test Accuracy=", round(accuracy_score(y_test, y_pred_gb), 2))

Train Accuracy= 0.6744108306869463
Test Accuracy= 0.63


In [ ]:
print(confusion_matrix(y_test, y_pred_gb))

[[ 505  950    2]
 [ 115 2328    9]
 [  19  554    6]]


In [ ]:
print(classification_report(y_test, y_pred_gb))

              precision    recall  f1-score   support

           0       0.79      0.35      0.48      1457
           1       0.61      0.95      0.74      2452
           2       0.35      0.01      0.02       579

    accuracy                           0.63      4488
   macro avg       0.58      0.44      0.41      4488
weighted avg       0.63      0.63      0.56      4488



## XGBClassifier

In [ ]:
# Create an instance of the XGBClassifier
model_xgb = XGBClassifier(objective='multi:softprob')

In [ ]:
# Fit the model to the training data
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
# Predicting the Test set results
y_pred_xgb = model_xgb.predict(X_test)

In [ ]:
print ("Train Accuracy=",accuracy_score(y_train, model_xgb.predict(X_train)))
print ("Test Accuracy=", round(accuracy_score(y_test, y_pred_xgb), 2))

Train Accuracy= 0.7717978717477296
Test Accuracy= 0.63


In [ ]:
print(confusion_matrix(y_test, y_pred_xgb))

[[ 646  795   16]
 [ 250 2163   39]
 [  45  511   23]]


In [ ]:
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.69      0.44      0.54      1457
           1       0.62      0.88      0.73      2452
           2       0.29      0.04      0.07       579

    accuracy                           0.63      4488
   macro avg       0.53      0.46      0.45      4488
weighted avg       0.60      0.63      0.58      4488



## Model Saving

In [ ]:
X_test

array([[ 1.56815967,  1.15406529, -1.04934436, ..., -0.29886517,
        -0.97837206,  0.0946872 ],
       [-1.20135891,  1.15406529, -1.04934436, ..., -0.29886517,
        -0.09395526,  0.44165438],
       [-1.75526262, -0.70303224, -1.04934436, ..., -0.29886517,
         0.04166353,  0.5160045 ],
       ...,
       [-1.20135891, -1.63158101, -1.04934436, ..., -0.29886517,
        -0.66629839,  0.61513798],
       [-1.20135891,  1.15406529, -1.04934436, ..., -0.29886517,
        -0.04161846,  0.3425209 ],
       [-0.64745519,  1.15406529,  0.952976  , ..., -0.29886517,
         1.23517561,  0.60894213]])

In [ ]:
# save the model to disk
filename = 'model_xgb.sav'
pickle.dump(model_xgb, open(filename, 'wb'))
print('Successfully saving the model')

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6310160427807486
